# 03b: Classification Algorithms

## วัตถุประสงค์การเรียนรู้
- เข้าใจหลักการของ Classification algorithms
- รู้จัก Logistic Regression, Decision Trees, Random Forest
- เข้าใจ metrics สำหรับ Classification

---

## 1. Classification คืออะไร?

- การทำนายหมวดหมู่ (class/category) ของข้อมูล
- ผลลัพธ์เป็นค่าแบบแยกส่วน (discrete)
- **Binary**: 2 คลาส (ใช่/ไม่ใช่)
- **Multiclass**: หลายคลาส (A/B/C/D)

### ตัวอย่างการใช้งาน
- Email Spam Detection
- Medical Diagnosis
- Customer Segmentation
- Image Recognition

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# สร้างข้อมูลตัวอย่าง Binary Classification
np.random.seed(42)
X, y = make_classification(n_samples=300, n_features=2, n_redundant=0, 
                          n_informative=2, n_clusters_per_class=1, random_state=42)

# แสดงข้อมูล
plt.figure(figsize=(8, 6))
plt.scatter(X[y==0, 0], X[y==0, 1], c='red', marker='o', label='Class 0')
plt.scatter(X[y==1, 0], X[y==1, 1], c='blue', marker='s', label='Class 1')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Binary Classification Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"จำนวนข้อมูล: {X.shape[0]}")
print(f"จำนวน Features: {X.shape[1]}")
print(f"การกระจายของ Classes: {np.bincount(y)}")

## 2. Logistic Regression

### หลักการ
- ใช้ Sigmoid function เพื่อแปลงค่าเป็นความน่าจะเป็น (0-1)
- เหมาะสำหรับ Binary Classification
- ตีความได้ง่าย
- เร็วในการฝึกและทำนาย

### Sigmoid Function
σ(z) = 1 / (1 + e^(-z))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# แบ่งข้อมูล
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# สร้างและฝึก Logistic Regression
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)

# ทำนาย
y_pred_lr = lr_model.predict(X_test)
y_prob_lr = lr_model.predict_proba(X_test)[:, 1]  # ความน่าจะเป็นของ class 1

print("Logistic Regression Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))

In [ ]:
# แสดง Decision Boundary
def plot_decision_boundary(model, X, y, title):
    h = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.RdYlBu)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu, edgecolors='black')
    plt.title(title)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')

plt.figure(figsize=(10, 6))
plot_decision_boundary(lr_model, X, y, 'Logistic Regression Decision Boundary')
plt.show()

## 3. Decision Trees

### หลักการ
- สร้างต้นไม้การตัดสินใจ
- แบ่งข้อมูลตาม features ที่ให้ information gain สูงสุด
- ตีความได้ง่าย
- จัดการ categorical และ numerical data ได้

### ข้อดี/ข้อเสีย
- **ข้อดี**: ตีความง่าย, ไม่ต้อง preprocessing มาก
- **ข้อเสีย**: ง่ายต่อ overfitting

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

# สร้างและฝึก Decision Tree
dt_model = DecisionTreeClassifier(max_depth=3, random_state=42)
dt_model.fit(X_train, y_train)

# ทำนาย
y_pred_dt = dt_model.predict(X_test)

print("Decision Tree Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_dt):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_dt))

In [ ]:
# แสดงโครงสร้างของ Decision Tree
plt.figure(figsize=(15, 8))
plot_tree(dt_model, filled=True, feature_names=['Feature 1', 'Feature 2'], 
          class_names=['Class 0', 'Class 1'], fontsize=10)
plt.title('Decision Tree Structure')
plt.show()

# แสดง Decision Boundary
plt.figure(figsize=(10, 6))
plot_decision_boundary(dt_model, X, y, 'Decision Tree Decision Boundary')
plt.show()

## 4. Random Forest

### หลักการ
- Ensemble method ที่รวม Decision Trees หลายๆ ต้น
- แต่ละต้นใช้ข้อมูลย่อยและ features ย่อย
- ลดปัญหา overfitting
- ให้ผลลัพธ์ที่แม่นยำกว่า single tree

### ข้อดี
- ความแม่นยำสูง
- จัดการ missing values ได้
- ให้ feature importance

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# สร้างและฝึก Random Forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
rf_model.fit(X_train, y_train)

# ทำนาย
y_pred_rf = rf_model.predict(X_test)

print("Random Forest Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))

# Feature Importance
feature_importance = rf_model.feature_importances_
print("\nFeature Importance:")
for i, importance in enumerate(feature_importance):
    print(f"Feature {i+1}: {importance:.3f}")

In [ ]:
# เปรียบเทียบ Decision Boundaries ของแต่ละ algorithm
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

models = [lr_model, dt_model, rf_model]
titles = ['Logistic Regression', 'Decision Tree', 'Random Forest']

for i, (model, title) in enumerate(zip(models, titles)):
    plt.subplot(1, 3, i+1)
    plot_decision_boundary(model, X, y, title)

plt.tight_layout()
plt.show()

## 5. Classification Metrics

### หลัก Metrics
- **Accuracy**: ความถูกต้องโดยรวม
- **Precision**: ความแม่นยำของการทำนาย positive
- **Recall**: ความครอบคลุมของ positive cases
- **F1-Score**: ค่าเฉลี่ยของ Precision และ Recall

### Confusion Matrix
- แสดงผลการจำแนกแต่ละคลาส
- ช่วยวิเคราะห์ข้อผิดพลาด

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# คำนวณ metrics สำหรับแต่ละ model
models_results = {
    'Logistic Regression': y_pred_lr,
    'Decision Tree': y_pred_dt,
    'Random Forest': y_pred_rf
}

results_df = pd.DataFrame()

for name, predictions in models_results.items():
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    
    results_df[name] = [accuracy, precision, recall, f1]

results_df.index = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
print("Model Comparison:")
print(results_df.round(3))

In [ ]:
# แสดง Confusion Matrix
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, (name, predictions) in enumerate(models_results.items()):
    cm = confusion_matrix(y_test, predictions)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
    axes[i].set_title(f'{name}\nConfusion Matrix')
    axes[i].set_xlabel('Predicted')
    axes[i].set_ylabel('Actual')

plt.tight_layout()
plt.show()

## สรุป

### Classification Algorithms
- **Logistic Regression**: เร็ว, ตีความได้, เหมาะกับ linear data
- **Decision Tree**: ตีความง่าย, จัดการ non-linear ได้
- **Random Forest**: ความแม่นยำสูง, ลด overfitting

### Metrics
- **Accuracy**: สำหรับ balanced data
- **Precision/Recall**: สำหรับ imbalanced data
- **F1-Score**: สมดุลระหว่าง Precision และ Recall

### Next: Classification LAB